# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

# 고객별 백화점 구매기록 데이터셋
- 미혼(1) or 기혼(0) 고객을 예측
- 학습용 구매기록 데이터
    - https://drive.google.com/file/d/1tg41qXu02FK55bFa8P1Gx2URhwPUCoQQ/view?usp=sharing
- 학습용 정답 데이터
    - https://drive.google.com/file/d/1-9AcU9nAoO4SzSmqdCOYTvWNpA5Pdt3q/view?usp=sharing
- 테스트용 구매기록 데이터
    - https://drive.google.com/file/d/1-AwDfGlHm9rNtpnHIWOK96jBJYy3f2SZ/view?usp=sharing
- 제출 양식 데이터
    - https://drive.google.com/file/d/1-Qv7SlsY5Eu3bRR7Z0IUJyLkOO1Fl3y6/view?usp=sharing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수

In [2]:
DATA_PATH_1 = "/content/drive/MyDrive/data/"
DATA_PATH_2="/content/drive/MyDrive/final_pj/"

- 데이터 불러오기

In [24]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH_1}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH_1}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH_1}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH_1}store_submission.csv") # 제출 양식 데이터

test_ft = pd.read_csv(f"{DATA_PATH_2}test_common_v1.1_1101.csv")
train_ft = pd.read_csv(f"{DATA_PATH_2}train_common_v1.1_1101.csv")
train_tr.shape , train_target.shape , test_tr.shape , submit.shape, test_ft.shape, train_ft.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2), (12225, 441), (14940, 441))

In [25]:
train_tr.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000


In [26]:
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,공휴일_대분류수정_의류_구매금액,공휴일_가정용품_대분류수정_구매횟수,공휴일_명품_대분류수정_구매횟수,공휴일_생식품_대분류수정_구매횟수,공휴일_생활용품_대분류수정_구매횟수,공휴일_아동_대분류수정_구매횟수,공휴일_영_대분류수정_구매횟수,공휴일_의류_대분류수정_구매횟수,대분류_수정_평균금액최대,대분류_수정_평균구매가격
0,train_0,11,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,가정용품,421000.000000
1,train_1,21,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,175000.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,명품,193500.000000
2,train_2,56,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,82000.0,0.0,7.0,0.0,0.0,1.0,0.0,1.0,명품,332597.058824
3,train_3,92,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,1198800.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,의류,233178.666667
4,train_4,27,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,1163000.0,0.0,2.0,0.0,5.0,3.0,0.0,3.0,명품,86240.000000


## 날짜 형식으로 변환

In [27]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

## 지점을 이용한 특성생성

In [29]:
agg_list = [

    ('저가구매_횟수', lambda x: ((x > 0) & (x <= 28000)).sum()),
    ('중저가구매_횟수', lambda x: ((x > 28000) & (x <= 58000)).sum()),
    ('중고가구매_횟수', lambda x: ((x > 58000) & (x <= 128000)).sum()),
    ('고가구매_횟수', lambda x: ((x > 128000) & (x <= 30820000)).sum())
]

tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")


((14940, 445), (12225, 445))

In [ ]:
tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,지점다양성_비율
0,test_0,0.142857
1,test_1,0.500000
2,test_2,0.200000
3,test_3,0.031250
4,test_4,0.142857


## 구매가격을 이용한 특성생성

# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [ ]:
train_ft.shape, test_ft.shape

((14940, 10), (12225, 10))

# 추출한 피처 저장하기

In [ ]:
train_ft.to_csv(f"{DATA_PATH_2}train_common.csv",index=False)
test_ft.to_csv(f"{DATA_PATH_2}test_common.csv",index=False)